# Description

Create the ETL process to transfer data from the raw format to a working relational database

The database used here is a MYSQL database initialized using DOCKER.

To create and initialize the database:
- docker pull mysql:latest
- docker run --name=user_mysql_1 --env="MYSQL_ROOT_PASSWORD=root_password" -p 3306:3306 -d mysql:latest

To run the database:
- docker exec -it user_mysql_1 mysql -uroot -p

Inside the database:
- CREATE DATABASE test_db;
- CREATE USER 'newuser'@'%' IDENTIFIED BY 'newpassword';
- GRANT ALL PRIVILEGES ON test_db.* to 'newuser'@'%';

Reference
https://medium.com/swlh/how-to-connect-to-mysql-docker-from-python-application-on-macos-mojave-32c7834e5afa

# Import libraries and define functions and paths

## libraries

In [1]:
import pandas as pd
import datetime
import os

## functions

## Paths

In [2]:
if 'mendes' in os.path.abspath(os.getcwd()):
    path_root = '/Users/mendes/tech_interview/Kensho/'
else:
    path_root = ''

path_raw_data_folder = path_root+'data/raw/'

# Read Raw data

In [3]:
data = pd.read_csv(path_raw_data_folder+'iris.csv')
data['timestamp'] = datetime.datetime.now()
data.head()

,sepal_length,sepal_width,petal_length,petal_width,species,timestamp
0,5.1,3.5,1.4,0.2,setosa,2020-07-18 11:26:08.688525
1,4.9,3.0,1.4,0.2,setosa,2020-07-18 11:26:08.688525
2,4.7,3.2,1.3,0.2,setosa,2020-07-18 11:26:08.688525
3,4.6,3.1,1.5,0.2,setosa,2020-07-18 11:26:08.688525
4,5.0,3.6,1.4,0.2,setosa,2020-07-18 11:26:08.688525


# Connect SQL DB and transfer data

## Define connection parameters

In [4]:
import sqlalchemy as db

# specify database configurations
config = {
    'host': 'localhost',
    'port': 3306,
    'user': 'mendes',
    'password': 'test',
    'database': 'test_db'
}
db_user = config.get('user')
db_pwd = config.get('password')
db_host = config.get('host')
db_port = config.get('port')
db_name = config.get('database')
# specify connection string
connection_str = f'mysql+pymysql://{db_user}:{db_pwd}@{db_host}:{db_port}/{db_name}'
# connect to database
engine = db.create_engine(connection_str)

In [5]:
# import pyodbc
# print("About to insert Hagi:")
# #Add your own SQL Server IP address, PORT, UID, PWD and Database
# conn = pyodbc.connect(
#     'DRIVER={FreeTDS};SERVER=**;PORT=**;DATABASE=**;UID=**;PWD=**', autocommit=True)
# cur = conn.cursor()
# #This is just an example
# cur.execute(
#     f"INSERT INTO [FootballPLayers] ([Name],[Age],[Job],[Country],[Married],[YearsEmployed]) VALUES ('Gheorghe Hagi','55','Manager','Romania','Y','6')")
# conn.commit()
# print('Should have inserted Hagi')
# cur.close()
# conn.close()

## Create or append to table

In [6]:
with engine.connect() as conn:
# pull metadata of a table
    metadata = db.MetaData(bind=engine)
    metadata.reflect(only=['test_table'])

    test_table = metadata.tables['test_table']
    frame = data.to_sql('iris',conn, if_exists='append');